# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
%load_ext dotenv
%dotenv 

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob
import sys


src_dir = os.getenv("SRC_DIR")
if src_dir and src_dir not in sys.path:
    sys.path.append(src_dir)

try:
    from utils.logger import get_logger
    _logs = get_logger(__name__)
except Exception:
    import logging
    logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
    _logs = logging.getLogger(__name__)
    _logs.warning("utils.logger.get_logger not found; using basic logging.")

_logs.info("dotenv loaded.")


2025-09-28 14:08:24,011, 908091552.py, 19, INFO, dotenv loaded.


In [4]:
from pathlib import Path

PRICE_DATA = os.getenv("PRICE_DATA")
assert PRICE_DATA, "Set PRICE_DATA in your .env (path to directory with parquet files)."

price_dir = Path(PRICE_DATA)
assert price_dir.exists(), f"PRICE_DATA does not exist: {price_dir}"

parquet_files = glob(os.path.join(PRICE_DATA, "**", "*.parquet"), recursive=True)
assert parquet_files, f"No parquet files found under {PRICE_DATA}"

_logs.info(f"Found {len(parquet_files)} parquet files.")
parquet_files[:5]


2025-09-28 14:08:24,287, 1684228962.py, 12, INFO, Found 3179 parquet files.


['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet']

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [5]:
import dask.dataframe as dd

dd_px = dd.read_parquet(parquet_files, engine="pyarrow")

rename_map = {}
if "Adj Close" in dd_px.columns: rename_map["Adj Close"] = "Adj_Close"
if "adj_close" in dd_px.columns and "Adj_Close" not in dd_px.columns: rename_map["adj_close"] = "Adj_Close"
if "close" in dd_px.columns and "Close" not in dd_px.columns: rename_map["close"] = "Close"
if "high"  in dd_px.columns and "High"  not in dd_px.columns: rename_map["high"]  = "High"
if "low"   in dd_px.columns and "Low"   not in dd_px.columns: rename_map["low"]   = "Low"
if "ticker" in dd_px.columns and "Ticker" not in dd_px.columns: rename_map["ticker"] = "Ticker"
if "date"   in dd_px.columns and "Date"   not in dd_px.columns: rename_map["date"]   = "Date"
dd_px = dd_px.rename(columns=rename_map)

if "Date" in dd_px.columns:
    dd_px = dd_px.set_index("Date", sorted=False)

dd_feat = dd_px.assign(
    Close_lag_1      = dd_px["Close"].shift(1),
    Adj_Close_lag_1  = (dd_px["Adj_Close"].shift(1) if "Adj_Close" in dd_px.columns else dd_px["Close"].shift(1)),
    returns          = dd_px["Close"] / dd_px["Close"].shift(1) - 1,
    hi_lo_range      = dd_px["High"] - dd_px["Low"]
)

dd_feat


,Open,High,Low,Close,Adj_Close,Volume,source,Ticker,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
npartitions=3179,,,,,,,,,,,,,
1973-02-22,float64,float64,float64,float64,float64,float64,string,string,int32,float64,float64,float64,float64
1973-08-06,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-30,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-01,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [6]:
import pandas as pd

df_feat = dd_feat.compute()

ticker_col = "ticker" if "ticker" in df_feat.columns else ("Ticker" if "Ticker" in df_feat.columns else None)
date_col   = "date"   if "date"   in df_feat.columns else ("Date"   if "Date"   in df_feat.columns else None)

if ticker_col is None:
    raise KeyError(f"No ticker column found. Columns present: {list(df_feat.columns)}")
if "returns" not in df_feat.columns:
    raise KeyError(f"'returns' column missing. Columns present: {list(df_feat.columns)}")

if ticker_col not in df_feat.columns:
    df_feat = df_feat.reset_index()

if date_col:
    df_feat = df_feat.sort_values([ticker_col, date_col])
else:
    df_feat = df_feat.sort_values([ticker_col])

df_feat["ret_ma_10"] = (
    df_feat.groupby(ticker_col, group_keys=False)["returns"]
           .transform(lambda s: s.rolling(10, min_periods=1).mean())
)

df_feat.head(10)


,Open,High,Low,Close,Adj_Close,Volume,source,Ticker,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,ret_ma_10
Date,,,,,,,,,,,,,,
2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,ACN,2001,29.299999,9.704098,-0.482253,0.29,-0.482253
2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,ACN,2001,20.750000,20.149536,-0.276626,0.25,-0.379440
2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,ACN,2001,13.570000,5.743644,0.105380,0.46,-0.217833
2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,ACN,2001,50.650002,25.081879,-0.706614,0.27,-0.340028
2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,ACN,2001,13.770000,10.636262,0.085693,0.30,-0.254884
2001-07-26,14.95,14.99,14.50,14.50,10.900705,6335300.0,ACN.csv,ACN,2001,34.080002,26.847973,-0.574531,0.49,-0.308158
2001-07-27,14.51,14.59,14.50,14.51,10.908223,3524000.0,ACN.csv,ACN,2001,13.600000,5.756337,0.066912,0.09,-0.254577
2001-07-30,14.50,14.78,14.50,14.70,11.051059,3654300.0,ACN.csv,ACN,2001,50.730000,25.121481,-0.710231,0.28,-0.311534
2001-07-31,14.71,15.01,14.60,14.96,11.246520,1429000.0,ACN.csv,ACN,2001,15.660000,12.096141,-0.044700,0.41,-0.281885


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
No. Dask supports rolling operations too. However, groupby + rolling across partition boundaries needs careful partitioning/overlap, which is easy to get wrong.

+ Would it have been better to do it in Dask? Why?
If the dataset is too large for memory, staying in Dask is better for scalability (with proper repartitioning and map_overlap/groupby-rolling). For small/medium data, pandas is simpler and less error-prone.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.